In [15]:
import json
import urllib.request as urlreq
import time, datetime
import pandas as pd
import codecs
# import sys
# print(sys.version)

In [16]:
def get_stock_list(fname):
    """Open .csv file containing list of stocks, and their weights in the ETF"""
    stock_list = pd.read_csv(fname)
    return stock_list

In [17]:
def get_quotes(stock_list):
    """Get quote for each stock in list, return list of responses from the API"""
    url = "http://finance.google.com/finance/info?client=ig&q="
    for stock in stock_list:
        url = url + '{},'.format(stock)
    raw_data = urlreq.urlopen(url).read().decode('utf8')

    return json.loads(raw_data[3:])

In [18]:
def calc_inav(stock_list, dow_divisor):
    """Calculate Indicative Net Asset Value of DJT. Returns calculated value, and value for
    the DJT received from API call"""
    quotes = get_quotes(stock_list['Ticker'])

    inav_df = stock_list.set_index('Ticker')

    for i, stock in enumerate(quotes):
        inav_df.set_value(stock['t'], 'Last_Price', float(stock['l'].replace(',','')))

    inav = inav_df[(inav_df.index != 'DJT')]['Last_Price'].sum(axis=0) / dow_divisor

    market_index = inav_df[(inav_df.index == 'DJT')]['Last_Price'].iloc[0]

    output = {'time': datetime.datetime.now(), 'inav': inav, 'market_index': market_index}

    return output

In [19]:
def plot():
    pass

In [20]:
def run_app(stock_list):
    """Will run the program continuously until user exits, getting new data every 1 secs"""
    try:
        while True:
            values = calc_inav(stock_list, dow_divisor)
            print('Calculated iNAV:', values['inav'], '\t', 
                  'Market Index:', values['market_index'],
                 '\t', 'Difference:', values['inav'] - values['market_index'])
            
            time.sleep(1)            

    except KeyboardInterrupt:
        print('Exiting...')
        exit()

In [21]:
if __name__ == '__main__':

    '''DOW divisor source: http://online.barrons.com/mdc/public/page/9_3022-djiahourly.html?mod=mdc_h_usshl'''
    dow_divisor = 0.16416809180007

    stock_list = get_stock_list('DJT_Component_Weights.csv')

    """Runs the program continuously until exit"""
    run_app(stock_list)

Calculated iNAV: 9458.84174551475 	 Market Index: 9457.25 	 Difference: 1.59174551475
Calculated iNAV: 9458.84174551475 	 Market Index: 9457.29 	 Difference: 1.55174551475
Calculated iNAV: 9458.719919161158 	 Market Index: 9457.41 	 Difference: 1.30991916116
Calculated iNAV: 9458.537179630772 	 Market Index: 9457.04 	 Difference: 1.49717963077
Calculated iNAV: 9458.476266453976 	 Market Index: 9457.04 	 Difference: 1.43626645397
Calculated iNAV: 9458.29352692359 	 Market Index: 9456.98 	 Difference: 1.31352692359
Calculated iNAV: 9458.354440100386 	 Market Index: 9456.68 	 Difference: 1.67444010039
Calculated iNAV: 9458.354440100386 	 Market Index: 9456.53 	 Difference: 1.82444010038
Calculated iNAV: 9458.354440100386 	 Market Index: 9456.5 	 Difference: 1.85444010039
Calculated iNAV: 9458.415353277182 	 Market Index: 9456.53 	 Difference: 1.88535327718
Calculated iNAV: 9458.415353277182 	 Market Index: 9456.47 	 Difference: 1.94535327718
Calculated iNAV: 9458.415353277182 	 Market Ind